In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from sqlalchemy import create_engine

## Netflix

In [2]:
csv_file = "source_data/netflix.csv"
netflix_df = pd.read_csv(csv_file)

In [3]:
netflix_df = netflix_df.loc[(netflix_df["Series or Movie"] == "Movie") & netflix_df["Country Availability"].str.contains("United States", na=False)]

In [4]:
netflix_df["Release_Year"] = pd.to_datetime(netflix_df["Release Date"], format="%d %b %Y").dt.year.fillna(0).astype(int)

In [5]:
netflix_df["Genre"] = netflix_df["Genre"].str.split(",").str[0]

In [6]:
netflix_movies = netflix_df[["Title", "Release_Year", "Genre"]].sort_values(by=["Title"], key=lambda col: col.str.lower()).reset_index(drop=True).fillna("None")
netflix_movies["Source"] = "Netflix"
netflix_movies.index.name = "netflix_id"

In [7]:
netflix_ratings = netflix_df[["Title", "IMDb Score", "Rotten Tomatoes Score", "Metacritic Score"]].sort_values(by=["Title"], key=lambda col: col.str.lower()).reset_index(drop=True).fillna("None")
netflix_ratings.columns = netflix_ratings.columns.str.replace(" Score", "_Rating")
netflix_ratings.rename(columns={"Rotten Tomatoes_Rating": "RT_Rating"}, inplace=True)
netflix_ratings.index.name = "netflix_id"

## Prime Video

In [8]:
csv_file = "source_data/prime_video.csv"
prime_video_df = pd.read_csv(csv_file)

In [9]:
prime_video_df = prime_video_df.loc[(prime_video_df["Language"] == "English")]

In [10]:
prime_video_movies = prime_video_df[["Movie Name", "Year of Release"]].sort_values(by=["Movie Name"], key=lambda col: col.str.lower()).reset_index(drop=True)
prime_video_movies["Year of Release"] = prime_video_movies["Year of Release"].str.replace("None", "0").astype(int)
prime_video_movies["Source"] = "Prime Video"
prime_video_movies.rename(columns={"Movie Name": "Title", "Year of Release": "Release_Year"}, inplace=True)
prime_video_movies.index.name = "prime_video_id"

In [11]:
prime_video_ratings = prime_video_df[["Movie Name", "IMDb Rating"]].sort_values(by=["Movie Name"], key=lambda col: col.str.lower()).reset_index(drop=True)
prime_video_ratings.rename(columns={"Movie Name": "Title"}, inplace=True)
prime_video_ratings.index.name = "prime_video_id"

## IMDb

In [12]:
csv_file = "source_data/imdb.csv"
imdb_df = pd.read_csv(csv_file, dtype={"year": str})

In [13]:
imdb_df = imdb_df[imdb_df["year"].str.len() == 4].reset_index(drop=True)

In [14]:
imdb_df["Release Year"] =  imdb_df["year"].astype(int)

In [15]:
imdb_df = imdb_df.loc[imdb_df["country"].str.contains("USA", na=False)]

In [16]:
imdb_df["Genre"] = imdb_df["genre"].str.split(",").str[0]

In [17]:
imdb_movies = imdb_df[["original_title", "Release Year", "Genre", "duration"]].sort_values(by=["original_title"], key=lambda col: col.str.lower()).reset_index(drop=True)
imdb_movies.rename(columns={"original_title": "Title", "duration": "Duration"}, inplace=True)
imdb_movies.index.name = "imdb_id"

In [18]:
imdb_ratings = imdb_df[["original_title", "avg_vote"]].sort_values(by=["original_title"], key=lambda col: col.str.lower()).reset_index(drop=True)
imdb_ratings.rename(columns={"original_title": "Title", "avg_vote": "IMDb_Rating"}, inplace=True)
imdb_ratings.index.name = "imdb_id"

## Rotten Tomatoes

In [19]:
csv_file = "source_data/rotten_tomatoes.csv"
tomatoes_df = pd.read_csv(csv_file)

In [20]:
tomatoes_df["Release Year"] = pd.to_datetime(tomatoes_df["original_release_date"], format="%Y-%m-%d").dt.year.fillna(0).astype(int)

In [21]:
tomatoes_df["Genre"] = tomatoes_df["genres"].str.split(",").str[0].str.split(" &").str[0]
tomatoes_movies = tomatoes_df[["movie_title", "Release Year", "Genre", "runtime"]].sort_values(by=["movie_title"], key=lambda col: col.str.lower()).reset_index(drop=True).fillna("None")
tomatoes_movies["Genre"] = tomatoes_movies["Genre"].str.replace("Science Fiction", "Sci-Fi")
tomatoes_movies.rename(columns={"movie_title": "Title", "runtime": "Duration"}, inplace=True)
tomatoes_movies.index.name = "tomatoes_id"

In [22]:
tomatoes_ratings = tomatoes_df[["movie_title", "tomatometer_rating", "audience_rating"]].sort_values(by=["movie_title"], key=lambda col: col.str.lower()).reset_index(drop=True)
tomatoes_ratings.rename(columns={"movie_title": "Title", "tomatometer_rating": "RT_Rating", "audience_rating": "RT_Audience_Rating"}, inplace=True)
tomatoes_ratings["RT_Rating"] = tomatoes_ratings["RT_Rating"].fillna(0).astype(int)
tomatoes_ratings["RT_Audience_Rating"] = tomatoes_ratings["RT_Audience_Rating"].fillna(0).astype(int)
tomatoes_ratings.index.name = "tomatoes_id"

## Connecting to SQLite database

In [23]:
engine = create_engine(f"sqlite:///movie_db.sqlite")

In [24]:
df_list = [netflix_movies, netflix_ratings, prime_video_movies, prime_video_ratings, imdb_movies, imdb_ratings, tomatoes_movies, tomatoes_ratings]
df_names = ["netflix_movies", "netflix_ratings", "prime_video_movies", "prime_video_ratings", "imdb_movies", "imdb_ratings", "tomatoes_movies", "tomatoes_ratings"]

In [25]:
for df, name in zip(df_list, df_names):
    df.to_sql(name=name, con=engine, if_exists="replace", index=True)

In [26]:
# engine.table_names()

In [27]:
# pd.read_sql_query("select * from netflix_movies", con=engine).head()